In [1]:
import pandas as pd
import numpy as np
import pickle

Let's build the FAR and FRR and analyze the best threshold to use for this model

# FAR

We need the test dataset when those values are Fraud.

In [ ]:
a = []

# get 100 X_test, where we have just fraud=1, at random
for i in range(100):
    # and get the probability of been fraud=0
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)
    class_index = yhat_class[0]
    class_probability = yhat_prob[0, class_index] * 100
    intp = int((class_probability))
    # print(f'Predicted: {intp} %')
    a.append(intp)

# apply the threshold and save how many observations passes
far = []
threshold = []
for i in range(100):
    num = 0
    for value in a:
        if value > i:
            num += 1
    far.append(num)
    threshold.appedn(i)

far = np.array(far)
far